# Reduction for the TRT data

The script to work with the TRT data. This code will perform the reduction and susequently aperture photometry on the TRT data.

Following packages are necessary to import:

In [11]:
import numpy as np
import matplotlib as mp
import ccdproc,os,sys,time,random
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from glob import glob
#from astroquery.astrometry_net import AstrometryNet
from astropy.wcs import WCS
import astroalign as aa
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.visualization import ImageNormalize, LogStretch
from matplotlib.ticker import LogLocator
from astropy.stats import SigmaClip, mad_std
import os

# 1 Aligning the science frames using astroalign for all the bands.

In [49]:
image_path='/mnt/eac99553-b108-449e-bf41-0716c975df8b/TRT-data/NGC 5947/band_wise/I_band/'
calibration_path='/mnt/eac99553-b108-449e-bf41-0716c975df8b/TRT-data/calibration/'
filename='*I.fits'
files=sorted(glob(os.path.join(image_path,filename)))
print(len(files))

ref_image=fits.open(files[2])
reference_image=ref_image[0].data

for i in range(0,1):
    image_data=fits.open(files[i])
    source_image=image_data[0].data
    header=image_data[0].header
    image_aligned,footprint=aa.register(source_image,reference_image)

    aligned_file=files[i].replace('.fits','')
    fits.writeto(aligned_file+'_aligned'+'.fits',image_aligned,header,overwrite=True)
    #os.system("mv *aligned* aligned/")
    print('No. %i alignment done'%i)

80


MaxIterError: Max iterations exceeded while trying to find acceptable transformation.

# 3. Aligning the images

Astro-Align package is used to alighn the images


INPUT:

    path: The directory where the images are kept (string)
    filename: The first few characters and the extension of the images (string). Example:
    j0946*fits, HD1104*fits etc.
    ref_image: Reference image which will be used to align the images.

OUTPUT:

    aligned images.


In [14]:
filename="*sliced*fits"
ref_image="j0947_i_1_sliced.fits"
nfiles=sorted(glob(path+filename))
image_data=fits.open(path+ref_image)
reference_image=image_data[0].data
for i in range(len(nfiles)):
    image_data=fits.open(nfiles[i])
    source_image=image_data[0].data
    header=image_data[0].header
    image_aligned,footprint=aa.register(source_image,reference_image)

    aligned_file=nfiles[i].replace('.fits','')
    fits.writeto(aligned_file+'_aligned'+'.fits',image_aligned,header,overwrite=True)

    print('No. %i alignment done'%i)


No. 0 done
No. 1 done
No. 2 done
No. 3 done
No. 4 done
No. 5 done
No. 6 done
No. 7 done
No. 8 done
No. 9 done
No. 10 done
No. 11 done
No. 12 done
No. 13 done
No. 14 done
No. 15 done
No. 16 done
No. 17 done
No. 18 done
No. 19 done
No. 20 done
No. 21 done
No. 22 done
No. 23 done
No. 24 done


# Cleaning the images

This module is meant for cleaning the images. The tasks to be included are: bias correction, flat correction, trimming, overscan as well as the cosmic ray removal from the science cases. (For the time we are skipping the overscan and trimming part.)

INPUT:

    path: The directory where the images are kept (string)
    filename: The first few characters and the extension of the images (string). Example:
    j0946*fits, HD1104*fits etc.

OUTPUT:


    cleaned images in the new directory: path/cleaned

# a) BIAS correction.

In [21]:
gain = 2 * u.electron / u.adu  # gain and readout noise are properties of the CCD and will change for different CCDs.
readnoise = 7.5 * u.electron

#ra=input('Enter the RA of the source:   ')
#dec=input('Enter the DEC of the source: ')


bias_files = sorted(glob(os.path.join(path,'bias*.fits')))
biaslist = []
for i in range (0,len(bias_files)):
    data= ccdproc.CCDData.read(bias_files[i],unit='adu')
        #data = ccdproc.create_deviation(data, gain=gain, readnoise=readnoise)
        #data= data-(data.uncertainty.array)
    biaslist.append(data)
    
masterbias = ccdproc.combine(biaslist,method='average',sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std)
masterbias.write('masterbias.fits', overwrite=True)
mbias=ccdproc.CCDData.read('masterbias.fits',unit='adu')
print('Master bias generated')
print(" Mean and median of the masterbias: ",np.mean(masterbias), np.median(masterbias))




INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Master bias generated
 Mean and median of the masterbias:  515.4987641731899 515.5


# b) Flat correction.

In [22]:
flat_files=sorted(glob(os.path.join(dir,'flat*.fits')))
flatlist = []
for j in range(0,len(flat_files)):
    flat=ccdproc.CCDData.read(flat_files[j],unit='adu')
    flat_bias_removed=ccdproc.subtract_bias(flat,masterbias)
    flatlist.append(flat_bias_removed)

    def inv_median(a):
        return 1 / np.median(a)

masterflat = ccdproc.combine(flatlist,method='median', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std)
masterflat.write('masterflat.fits', overwrite=True)
mflat=ccdproc.CCDData.read('masterflat.fits',unit='adu')
print('Master flat generated')
print(" Mean and median of the masterflat: ",np.mean(masterflat), np.median(masterflat))



INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Master flat generated
 Mean and median of the masterflat:  0.9992749321566833 0.9999320144129445


# c) Cosmic Ray removal.

In [ ]:
filename="*aligned*fits"
file_names = sorted(glob(os.path.join(dir,filename)))
for i in range(0,len(file_names)):
    image=ccdproc.CCDData.read(file_names[i],unit='adu')
    header=fits.getheader(file_names[i],0)
    bias_subtracted = ccdproc.subtract_bias(image, masterbias)
    flat_corrected = ccdproc.flat_correct(bias_subtracted, masterflat)
    cr_cleaned = ccdproc.cosmicray_lacosmic(flat_corrected,readnoise=7.5, sigclip=5,satlevel=65535,niter=20,cleantype='meanmask',gain_apply=True)
        #print('Cosmic rays removed')
    clean_file=file_names[i].replace('.fits','')
    fits.writeto(clean_file+'_cleaned.fits',cr_cleaned,header,overwrite=True)
    print('Image no-%i has been cleaned'%i)